## Word2vec, или о практике создания векторных моделей языка

*Тут будет вступление.*
- *о word2vec*
- *o реализации word2vec в библиотеке gensim*

Тьюториал состоит из трех частей:
* в первой части мы научимся осуществлять предобработку текстовых файлов и тренировать векторную модель.
* во второй части мы научимся загружать векторные модели и работать с ними, например, выполнять простые операции над векторами слов, такие как "сложить вектора двух слов" или "вычислить коэффициент близости между двумя векторами слов". 
* в третьей части мы рассмотрим более сложные операции над векторами, например, "найти семантические аналоги" или "найти лишний вектор".

Мы рекомендуем использовать **Python3**, работоспособность тьюториала для **Python2** не гарантируется.

## 1. Предобработка текстовых данных и тренировка модели

*Сказать о том, что предобрабатывать данные не обязательно, можно просто разделить на предложения, но мы сделаем более глубокую предобработку для совместимости с моделями RusVectores. Ну и просто потому что полезно*

### Обработка текста

Предобработка текстов для тренировки моделей выглядит следующим образом:
* сначала мы приведем все слова к начальной форме (лемматизируем) и удалим стоп-слова;
* затем мы приведем все леммы к нижнему регистру;
* для каждого слова добавим его частеречный тэг.

Давайте попробуем воссоздать процесс предобработки текста на примере рассказа [О. Генри "Русские соболя"](https://rusvectores.org/static/henry_sobolya.txt). Для предобработки можно использовать различные тэггеры, мы сейчас будем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe), чтобы сразу получить частеречную разметку в виде Universal POS-tags. Сначала установим обертку *UDPipe* для Python:

`pip install ufal.udpipe`

*UDPipe* использует предобученные модели для лемматизации и тэггинга. Вы можете использовать [уже готовую модель](https://rusvectores.org/static/models/udpipe_syntagrus.model) или обучить свою. 

Чтобы загружать файлы, можно использовать реализацию wget в виде питоновской библиотеки:

`pip install wget`

In [4]:
!pip install wget
!pip install ufal.udpipe

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=11803e6beea9c5354e9e189b0c391e34adf1ce845ca81d2c0155fe27217fe3fe
  Stored in directory: c:\users\arcry\appdata\local\pip\cache\wheels\a1\b6\7c\0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [5]:
import wget
import sys

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

100% [..............................................................................] 25649 / 25649

Приступим к собственно предобработке текста. Попробуем лемматизировать текст и добавить частеречные тэги при помощи этой функции:

In [6]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


Эту функцию можно также изменить под конкретную задачу. Например, если частеречные тэги нам не нужны, в функции ниже выставим `keep_pos=False`. Если необходимо сохранить знаки пунктуации, можно выставить `keep_punct=True`. 

Теперь загружаем модель *UDPipe*, читаем текстовый файл и обрабатываем его при помощи нашей функции. В файле должен содержаться необработанный текст (одно предложение на строку или один абзац на строку).
Этот текст токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход мы получаем последовательность разделенных пробелами лемм с частями речи ("зеленый\_NOUN трамвай\_NOUN").

In [7]:
from ufal.udpipe import Model, Pipeline
import os
import re

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
    return '\n'.join(tagged)

In [8]:
text = open(textfile, 'r', encoding='utf-8').read()
processed_text = tag_ud(text=text, modelfile=modelfile)
print(processed_text[:350])
with open('my_text.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)


Loading the model...
Processing input...


русский_PROPN  соболь_NOUN о.::генри_PROPN 
когда_SCONJ синий_ADJ как_SCONJ ночь_NOUN глаз_NOUN Молли_VERB Мак-Кивер_PROPN  класть_VERB малыш::Брэди_PROPN  на_ADP оба_NUM лопатка_NOUN он_PRON вынужденный_ADJ быть_AUX покидать_VERB ряд_NOUN банда_NOUN «Дымовый_ADJ труба»_NOUN таков_ADJ власть_NOUN нежный_ADJ укор_NOUN подружка_NOUN и_CCONJ она_PRON 


Наша функция напечатает обработанный текст, который мы теперь можем также сохранить в файл. 

Итак, в ходе этой части тьюториала мы научились от "сырого текста" приходить к лемматизированному тексту с частеречными тэгами, который уже можно подавать на вход модели! Теперь теперь попробуем натренировать векторную модель.

### Процесс тренировки модели 

Для работы с эмбеддингами слов существуют различные библиотеки: [gensim](https://radimrehurek.com/gensim/), [keras](https://keras.io/), [tensorflow](https://www.tensorflow.org/), [pytorch](https://pytorch.org/). Мы будем работать с библиотекой *gensim*, ведь в основе нашего сервера именно она и используется.


***Gensim***  - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для *topic modeling* в ней реализованы на python и алгоритмы из тулкита *word2vec* (который в оригинале был написан на C++). Прежде всего, если *gensim* у вас на компьютере не установлен, нужно его установить:

`pip install gensim`

Gensim регулярно обновляется, так что не будет лишним удостовериться, что у вас установлена последняя версия, а при необходимости проапдейтить библиотеку:

`pip install gensim --upgrade` 

или 

`pip install gensim -U`

При подготовке этого тьюториала использовался *gensim* версии 3.7.0.

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека `logging`.

In [9]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


На вход модели мы даем наш обработанный текстовый файл (либо любой другой текст, важно лишь, что каждое предложение должно быть на отдельной строчке).

In [18]:
f = 'C:/Users/Arcry/SkillBox/my_text.txt'
data = gensim.models.word2vec.LineSentence(f)

In [20]:
data

Инициализируем модель. Параметры в скобочках:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [21]:
model = gensim.models.Word2Vec(data, vector_size=500, window=10, min_count=2, sg=0)

2021-07-02 17:00:42,883 : INFO : collecting all words and their counts
2021-07-02 17:00:42,884 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-02 17:00:42,887 : INFO : collected 981 word types from a corpus of 2085 raw words and 65 sentences
2021-07-02 17:00:42,888 : INFO : Creating a fresh vocabulary
2021-07-02 17:00:42,890 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 252 unique words (25.68807339449541%% of original 981, drops 729)', 'datetime': '2021-07-02T17:00:42.890557', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2021-07-02 17:00:42,891 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 leaves 1356 word corpus (65.03597122302158%% of original 2085, drops 729)', 'datetime': '2021-07-02T17:00:42.891543', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать.

In [22]:
model.init_sims(replace=True)

c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  """Entry point for launching an IPython kernel.
2021-07-02 17:00:50,804 : WARNING : destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


Смотрим, сколько в модели слов:

In [24]:
print(len(model.wv.index_to_key))

252


И сохраняем!

In [25]:
model.save('my.model')

2021-07-02 17:01:34,315 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'my.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-07-02T17:01:34.315584', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2021-07-02 17:01:34,316 : INFO : not storing attribute cum_table
2021-07-02 17:01:34,322 : INFO : saved my.model


## 2. Работа с векторными моделями при помощи библиотеки Gensim

Для своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

Существуют несколько форматов, в которых могут храниться модели. Во-первых, данные могут храниться в нативном формате *word2vec*, при этом модель может быть бинарной или не бинарной. Для загрузки модели в формате *word2vec* в классе `KeyedVectors` (в котором хранится большинство относящихся к дистрибутивным моделям функций) существует функция `load_word2vec_format`, а бинарность модели можно указать в аргументе `binary` (внизу будет пример). Помимо этого, модель можно хранить и в собственном формате *gensim*, для этого существует класс `Word2Vec` с функцией `load`. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте. Наш код определяет тип модели по её расширению, но вообще файл с моделью может называться как угодно, жестких ограничений для расширения нет.

Давайте скачаем новейшую модель для русского языка, созданную на основе [Национального корпуса русского языка (НКРЯ)](http://www.ruscorpora.ru/), и загрузим в её в память. Распаковывать скачанный архив для обычных моделей не нужно, так как его содержимое прочитается при помощи специальной инструкции:

In [27]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

 99% [..................................................................... ] 484237312 / 484452285

2021-07-02 17:03:43,899 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>


100% [......................................................................] 484452285 / 484452285

2021-07-02 17:03:47,471 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (189193, 300) matrix of type float32 from <zipfile.ZipExtFile [closed]>', 'binary': True, 'encoding': 'utf8', 'datetime': '2021-07-02T17:03:47.471737', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'load_word2vec_format'}


Скажем, нам интересны такие слова (пример для русского языка):

In [28]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [29]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_NOUN
неделя_NOUN 0.7375996112823486
день_PROPN 0.706766664981842
месяц_NOUN 0.7037326097488403
час_NOUN 0.6643949747085571
утро_NOUN 0.6526745557785034
вечер_NOUN 0.6038411855697632
сутки_NOUN 0.5923080444335938
воскресенье_NOUN 0.5842781066894531
полдень_NOUN 0.5743687748908997
суббота_NOUN 0.534594714641571


ночь_NOUN
ночь_PROPN 0.8310785293579102
вечер_NOUN 0.718367874622345
рассвет_NOUN 0.6965948343276978
ночи_NOUN 0.6920219659805298
полночь_NOUN 0.670497715473175
ночь_VERB 0.6615265011787415
утро_NOUN 0.6263935565948486
ночной_ADJ 0.6024709343910217
полдень_NOUN 0.5835085511207581
сумерки_NOUN 0.5671443939208984


человек_NOUN
человек_PROPN 0.7850059866905212
человеческий_ADJ 0.5915265679359436
существо_NOUN 0.5736929774284363
народ_NOUN 0.5354466438293457
личность_NOUN 0.5296981930732727
человечество_NOUN 0.5282931327819824
человкъ_PROPN 0.5047001242637634
индивидуум_NOUN 0.5000404119491577
нравственный_ADJ 0.4972919821739197
потому_ADV 0.4929361939430237


семантика_NOUN
с

Находим косинусную близость пары слов:

In [30]:
print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.22025342


## 3. Более сложные операции над векторами

Найди лишнее!

In [31]:
print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

картофель_NOUN


Реши пропорцию!

In [32]:
print(model.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

гамбургер_NOUN


# Заключение